In [1]:
from functional import seq
from functional.streams import Sequence
import pandas as pd
import numpy as np
import os
from typing import Dict, List, Tuple, Optional
import random
from collections import namedtuple
from random import shuffle

In [2]:
ustawy_dir = "../ustawy"
ustawy_files = seq(os.listdir(ustawy_dir)).map(
    lambda filename: open(ustawy_dir + "/" + filename).read())

ustawy_files.size()

1180

In [3]:
good, bad = ustawy_files.partition(lambda x: "Art" in x[:2400])

In [4]:
good.size()

1178

In [5]:
bad.size()

2

In [6]:
bad[0][:300].split("\n")

['',
 'USTAWA',
 'z dnia 11 października 2013 r.',
 'o wzajemnej pomocy przy dochodzeniu podatków, należności ',
 'celnych i innych należności pieniężnych',
 '',
 '<!-/*',
 'Font Definitions */',
 ' @font-face',
 '\t{font-family:Helvetica;',
 '\tpanose-1:2 11 5 4 2 2 2 2 2 4;}',
 '@font-face',
 '\t{font-family:Courier;',
 '\tpanose-1:2 7 4 9 2 2 5 2 4 4;}',
 '']

In [7]:
bad[1][:300].split("\n")

['', '', '', '', '', 'Brak tekstu w postaci elektronicznej ', '']

In [8]:
def is_change(ustawa:str) -> bool:
    return "o zmianie ustawy" in ustawa[:800]

In [9]:
changes, not_changes = good.partition(is_change)

In [10]:
Labeled =  namedtuple("Labeled","text is_change")

In [11]:
def strip_title(text:str) -> str:
    return text.split("Art",maxsplit=1)[1]

In [12]:
labeled_changes = changes.map(lambda txt: Labeled(text = strip_title(txt), is_change = True))

In [13]:
labeled_not_changes = not_changes.map(lambda txt: Labeled(text = strip_title(txt), is_change = False))

In [14]:
data = (labeled_changes + labeled_not_changes).to_list()

In [15]:
shuffle(data)

In [18]:
def split(data):
    data_len = len(data)
    train_size = int(0.6 * data_len)
    validation_size = int(0.2 * data_len)

    train = data[:train_size]
    validation = data[train_size:train_size + validation_size]
    test = data[train_size + validation_size:]
    return train,test,validation
    
# train, test,validation = split(data)
# len(train),len(validation),len(test)

In [19]:
from random import choices # todo:bcm - take without replacements
def full_selector(text):
    return text

def percentage_selector(text):
    lines = text.split("\n")
    take_count = int(0.1* len(lines))
    return "\n".join(choices(lines,k= take_count))

def lines_selector(text):
    lines = text.split("\n")
    return "\n".join(choices(lines,k= 10))
    
def line_selector(text):
    lines = text.split("\n")
    lines_len = len(lines)
    return lines[random.randint(0,lines_len-1)]

In [20]:
Selector = namedtuple("Selector", "name selector")

In [21]:
selectors = seq([
    Selector(name = "full", selector = full_selector),
    Selector(name = "percentage", selector = percentage_selector),
    Selector(name = "lines", selector = lines_selector),
    Selector(name = "line", selector = line_selector),
])

## Learning things

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report


In [23]:
def grid_search(train_x, train_y, test_x, test_y,  parameters, pipeline):
    grid_search_tune = GridSearchCV(
        pipeline, parameters, cv=2, n_jobs=3, verbose=10)
    grid_search_tune.fit(train_x, train_y)

    print("Best parameters set:")
    print(grid_search_tune.best_estimator_.steps)

    print("Applying best classifier on test data:")
    best_clf = grid_search_tune.best_estimator_
    predictions = best_clf.predict(test_x)

    print(classification_report(test_y, predictions))

In [24]:
sw_file = open("polish.stopwords.txt")
stop_words = set(word[:-1] for word in sw_file.readlines())
sw_file.close()
list(stop_words)[:3]

['niż', 'pod', 'twój']

In [27]:
import nltk

tokenized = [Labeled(text = nltk.word_tokenize(x.text), is_change= x.is_change) for x in data]

In [30]:
def map_words(mapper, data):
    return [Labeled(text = mapper(x.text), is_change= x.is_change) for x in data]

In [32]:
# train_x = [x.text for x in train]
# y_train = [x.is_change for x in train]
# test_x = [x.text for x in test]
# y_test = [x.is_change for x in test]

In [57]:
def split2(x,y):
    data_len = len(y)
    train_size = int(0.6 * data_len)
    validation_size = int(0.2 * data_len)

    train = x[:train_size]
    validation = x[train_size:train_size + validation_size]
    test = x[train_size + validation_size:]
    
    y_train = y[:train_size]
    y_validation = y[train_size:train_size + validation_size]
    y_test = y[train_size + validation_size:]
    
    
    return (train,test,validation), (y_train,y_test,y_validation)

In [62]:
xs = [x.text for x in data]
ys = [x.is_change for x in data]

In [64]:
from collections import Counter
word_counter = Counter()

for labeled_doc in tokenized:
    word_counter.
    

In [63]:
idf = TfidfVectorizer(stop_words=stop_words)
idfed = idf.fit_transform(xs)

In [59]:
(tr_x,test_x,val_x),(tr_y,test_y,val_y)= split2(idfed,ys)

In [60]:
svc = SVC()#OneVsRestClassifier(SVC(), n_jobs=1)
svc.fit(tr_x,tr_y)

/Users/jakubdarul/Library/Python/3.7/lib/python/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [61]:
svc.score(test_x,test_y)

0.4641350210970464